In [1]:
import warnings
warnings.filterwarnings("ignore")

from xai_agg import *

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.ensemble import RandomForestClassifier

import pandas as pd
import numpy as np

import dill

2025-01-13 11:12:06.095159: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-01-13 11:12:06.122755: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
raw_data = pd.read_csv('../data/wdbc.csv')
# Specify header:
raw_data.columns = ['ID', 'Diagnosis'] + [str for i in range(1, 4) for str in [f"radius_{i}", f"texture_{i}", f"perimeter_{i}", f"area_{i}", f"smoothness_{i}", f"compactness_{i}", f"concavity_{i}", f"concave_points_{i}", f"symmetry_{i}", f"fractal_dimension_{i}"]]

display(raw_data.head())

,ID,Diagnosis,radius_1,texture_1,perimeter_1,area_1,smoothness_1,compactness_1,concavity_1,concave_points_1,...,radius_3,texture_3,perimeter_3,area_3,smoothness_3,compactness_3,concavity_3,concave_points_3,symmetry_3,fractal_dimension_3
0,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
1,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
2,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
3,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678
4,843786,M,12.45,15.70,82.57,477.1,0.12780,0.17000,0.1578,0.08089,...,15.47,23.75,103.40,741.6,0.1791,0.5249,0.5355,0.1741,0.3985,0.12440


In [3]:
preprocessed_data = raw_data.drop(columns=['ID'])

preprocessed_data['Diagnosis'] = preprocessed_data['Diagnosis'].map({'M': 1, 'B': 0})
preprocessed_data.rename(columns={'Diagnosis': 'DiagnosisIsMalignant'}, inplace=True)

categorical_features = []

In [4]:
X = preprocessed_data.drop(columns=['DiagnosisIsMalignant'])
y = preprocessed_data['DiagnosisIsMalignant']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [5]:
clf = RandomForestClassifier(random_state=42)
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

print(f"Accuracy: {accuracy_score(y_test, y_pred)}")
print(f"ROC AUC: {roc_auc_score(y_test, y_pred)}")

Accuracy: 0.9736842105263158
ROC AUC: 0.9673913043478262


# Experiments
# RAE-T vs. RAE-E | 10 samples
### Execution

In [ ]:
results, metadata = evaluate_aggregate_explainer(
    clf, X_train, X_test, categorical_features,
    metrics_sets=[['nrc', 'sensitivity_spearman', 'faithfulness_corr']],
    mcdm_algs=[pymcdm.methods.TOPSIS(), pymcdm.methods.EDAS()],
    n_instances=10
)

metadata["description"] = "RAE-T vs RAE-S, 10 samples"

with open('pickles/wdbc/RAE-T_vs_RAE-S_10.pkl', 'wb') as f:
    dill.dump(ExperimentRun(metadata, results), f)

Selected indexes: [494 526 460 182 204 153 148 556  30 537]
Epoch 1/500
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 1.2619 - val_loss: 1.2735
Epoch 2/500
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.2299 - val_loss: 1.2590
Epoch 3/500
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.2586 - val_loss: 1.2460
Epoch 4/500
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1.2508 - val_loss: 1.2331
Epoch 5/500
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.2000 - val_loss: 1.2189
Epoch 6/500
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.2055 - val_loss: 1.2025
Epoch 7/500
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.2318 - val_loss: 1.1831
Epoch 8/500
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.0905 - val_loss: 1.1599
Epoch 9/500
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.1930 - val_loss: 1.1337
Epoch 10/500
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.0069 - val_loss: 1.1058
Epoch 11/500
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.0953 - val_loss: 1.0760
Epo

### Analysis

In [9]:
with open('pickles/wdbc/RAE-T_vs_RAE-S_10.pkl', 'rb') as f:
    exp = dill.load(f)

In [10]:
methods = ["RAE-T", "RAE-E"]
for i, method in enumerate(methods):
    print(f"{method}:\n")
    display(exp.results[i])
    print(f"Worst case avoidances: {count_worst_case_avoidances(exp.results[i], [False, True, True], 1)}")
    print("AVG:")
    display(get_expconfig_mean_results(exp, i))
    print("\n")

RAE-T:



[                              nrc  sensitivity_spearman  faithfulness_corr
 LimeWrapper             52.569576              0.947942           0.340361
 ShapTabularTreeWrapper  46.275174              0.997775           0.368225
 AnchorWrapper           44.563923              0.423960           0.595710
 AggregateExplainer      54.043721              0.927164           0.609184,
                               nrc  sensitivity_spearman  faithfulness_corr
 LimeWrapper             50.744528              0.963871           0.374226
 ShapTabularTreeWrapper  51.548148              0.996796           0.464866
 AnchorWrapper           44.563923              0.494440           0.557977
 AggregateExplainer      51.582875              0.936285           0.138805,
                               nrc  sensitivity_spearman  faithfulness_corr
 LimeWrapper             55.156314              0.997241           0.263274
 ShapTabularTreeWrapper  52.654453              0.998710           0.783782
 AnchorWra

Worst case avoidances: [7, 9]
AVG:


,nrc,sensitivity_spearman,faithfulness_corr
AggregateExplainer,51.935090,0.942073,0.553357
AnchorWrapper,50.197264,0.367354,0.397601
LimeWrapper,52.063108,0.957188,0.339886
ShapTabularTreeWrapper,51.141907,0.996832,0.583134




RAE-E:



[                              nrc  sensitivity_spearman  faithfulness_corr
 LimeWrapper             49.333615              0.951057           0.320150
 ShapTabularTreeWrapper  46.275174              0.992659           0.187503
 AnchorWrapper           44.563923              0.446314           0.092996
 AggregateExplainer      48.308280              0.931301           0.279235,
                               nrc  sensitivity_spearman  faithfulness_corr
 LimeWrapper             52.028628              0.934149           0.054523
 ShapTabularTreeWrapper  51.548148              0.995729           0.375234
 AnchorWrapper           61.796491              0.545775           0.254731
 AggregateExplainer      52.439622              0.942692           0.523626,
                               nrc  sensitivity_spearman  faithfulness_corr
 LimeWrapper             54.668409              0.996663           0.498338
 ShapTabularTreeWrapper  52.654453              0.998977           0.955594
 AnchorWra

Worst case avoidances: [10, 10]
AVG:


,nrc,sensitivity_spearman,faithfulness_corr
AggregateExplainer,50.202641,0.950011,0.552567
AnchorWrapper,46.263853,0.373594,0.204732
LimeWrapper,52.711332,0.955724,0.419029
ShapTabularTreeWrapper,51.141907,0.995497,0.429594
